In [ ]:
import pandas as pd

test_results_validated = pd.read_csv('test_folds_validated.csv', sep = '\t')
bin_results = pd.read_csv('acsess_bin_results_folds_test_2.csv', sep = '\t')
total_adjusted = pd.read_csv('df_total_adjucated.csv', sep = '\t')
fold_5 = pd.read_csv('acsess_bin_results_folds_test_fold5only.csv', sep='\t')

In [ ]:
bin_results['validated'] = test_results_validated['validated']
bin_results['binary k=5'] = fold_5['binary k=5']
bin_results['binary k=2'] = [1] * len(bin_results)
bin_results['binary k=3'] = [0] * len(bin_results)
bin_results['binary k=4'] = [0] * len(bin_results)
bin_results.columns = bin_results.columns.str.replace('binary ', '', regex=False)

In [ ]:
bin_results[['k=0', 'k=1', 'k=5']] = bin_results[['k=0', 'k=1', 'k=5']].applymap(lambda x: 1 if x > 1 else x)

In [ ]:
bin_results.dropna()
print(bin_results.columns)

In [ ]:
bin_results = bin_results[bin_results['relevance_manual'] == bin_results['validated']]

In [ ]:
print(bin_results)
print(bin_results['note_nr'].unique())

In [ ]:
total_adjusted = total_adjusted[total_adjusted['note_nr'].isin(bin_results['note_nr'].unique())]

In [ ]:
print(total_adjusted.head())

In [ ]:
#grouped = bin_results.groupby('note_nr')
#grouped_bin = grouped.sum()

grouped = bin_results.groupby('note_nr').agg({'k=0': 'max', 'k=1': 'max', 'k=5': 'max'}).reset_index()


In [ ]:
grouped = grouped[['note_nr', 'k=0', 'k=1', 'k=5']]

In [ ]:
total_test = pd.merge(grouped, total_adjusted, on='note_nr', how='left')

In [ ]:
bin_results['k=0'].unique()

In [ ]:
print(total_test)

In [ ]:
print(bin_results)

In [ ]:
#Evaluation sentence level
bin_results.columns = bin_results.columns.str.replace('binary ', '', regex=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_metrics(df):
    # Initialize a dictionary to store the results
    results = {f'k={i}': {'precision': [], 'recall': [], 'f1_score': []} for i in range(0, 6)}

    # Loop over each fold and each k
    for fold in ['fold1', 'fold2', 'fold3', 'fold4', 'fold5']:
        for k in range(0, 6):
            # Filter the data for the current fold
            fold_data = df[df['fold'] == fold]

            # Get the ground truth and predictions, ignoring NaN values
            y_true = fold_data['relevance_manual']
            y_pred = fold_data[f'k={k}'].dropna()
            y_true = y_true[y_pred.index]

            # Calculate precision, recall, and F1-score
            precision = precision_score(y_true, y_pred, zero_division=0)
            recall = recall_score(y_true, y_pred, zero_division=0)
            f1 = f1_score(y_true, y_pred, zero_division=0)

            # Append the scores to the results dictionary
            results[f'k={k}']['precision'].append(precision)
            results[f'k={k}']['recall'].append(recall)
            results[f'k={k}']['f1_score'].append(f1)

    return results

def generate_results_table(df):
    # Calculate metrics
    results = calculate_metrics(df)

    # Calculate averages and standard deviations for each k
    precision_avg_list, precision_std_list = [], []
    recall_avg_list, recall_std_list = [], []
    f1_avg_list, f1_std_list = [], []

    for k in range(0, 6):
        precision_avg = np.mean(results[f'k={k}']['precision'])
        precision_std = np.std(results[f'k={k}']['precision'])
        recall_avg = np.mean(results[f'k={k}']['recall'])
        recall_std = np.std(results[f'k={k}']['recall'])
        f1_avg = np.mean(results[f'k={k}']['f1_score'])
        f1_std = np.std(results[f'k={k}']['f1_score'])

        precision_avg_list.append(precision_avg)
        precision_std_list.append(precision_std)
        recall_avg_list.append(recall_avg)
        recall_std_list.append(recall_std)
        f1_avg_list.append(f1_avg)
        f1_std_list.append(f1_std)

    # Create a DataFrame to store the results
    results_table = pd.DataFrame({
        'k': [f'k={i}' for i in range(0, 6)],
        'Precision (Avg)': precision_avg_list,
        'Precision (Std)': precision_std_list,
        'Recall (Avg)': recall_avg_list,
        'Recall (Std)': recall_std_list,
        'F1-score (Avg)': f1_avg_list,
        'F1-score (Std)': f1_std_list
    })

    return results_table

# Example of how to use the functions on a dataframe
print("ACSESS")
results_table1 = generate_results_table(bin_results)
print(results_table1.to_string(index=False))


In [ ]:

#results_table1 = generate_results_table(bin_results, 'relevance_manual', ['k=0', 'k=1', 'k=5'])
#print(results_table1.to_string(index=False))


In [ ]:
#total_test.dropna()
print(total_test['k=0'].unique())
print(total_test['k=1'].unique())
print(total_test['k=5'].unique())
#total_test[['k=0', 'k=1', 'k=5']] = total_test[['k=0', 'k=1', 'k=5']].applymap(lambda x: 1 if x > 1 else x)

In [ ]:
print(type(total_test))
print(total_test.columns)

In [ ]:
print(type(total_test['k=0'][0]))

In [ ]:
def float_to_int(value):
    if isinstance(value, float) and value.is_integer():
        return int(value)
    return value

#for col in ['k=0', 'k=1', 'k=5']:
total_test['k=0'] = total_test[col].apply(float_to_int)
total_test['k=1'] = total_test[col].apply(float_to_int)
total_test['k=5'] = total_test[col].apply(float_to_int)

print(type(total_test['k=0'][0]))    
    
#for col in ['k=0', 'k=1', 'k=5']:
#total_test = total_test[total_test[col].apply(lambda x: isinstance(x, int))]

In [ ]:
print(total_test.columns)

In [ ]:
pd.set_option('display.max_rows', None)
print(total_test['k=0'])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_metrics(df, target_column, prediction_columns):
    # Initialize a dictionary to store the results
    results = {col: {'precision': [], 'recall': [], 'f1_score': []} for col in prediction_columns}

    # Loop over each prediction column
    for col in prediction_columns:
        # Get the ground truth and predictions, ignoring NaN values
        y_true = df[target_column]
        y_pred = df[col].dropna()
        y_true = y_true.loc[y_pred.index]

        # Calculate precision, recall, and F1-score
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)

        # Append the scores to the results dictionary
        results[col]['precision'].append(precision)
        results[col]['recall'].append(recall)
        results[col]['f1_score'].append(f1)

    return results

def generate_results_table(df, target_column, prediction_columns):
    # Calculate metrics
    results = calculate_metrics(df, target_column, prediction_columns)

    # Calculate averages and standard deviations for each prediction column
    precision_avg_list, precision_std_list = [], []
    recall_avg_list, recall_std_list = [], []
    f1_avg_list, f1_std_list = [], []

    for col in prediction_columns:
        precision_avg = np.mean(results[col]['precision'])
        precision_std = np.std(results[col]['precision'])
        recall_avg = np.mean(results[col]['recall'])
        recall_std = np.std(results[col]['recall'])
        f1_avg = np.mean(results[col]['f1_score'])
        f1_std = np.std(results[col]['f1_score'])

        precision_avg_list.append(precision_avg)
        precision_std_list.append(precision_std)
        recall_avg_list.append(recall_avg)
        recall_std_list.append(recall_std)
        f1_avg_list.append(f1_avg)
        f1_std_list.append(f1_std)

    # Create a DataFrame to store the results
    results_table = pd.DataFrame({
        'Prediction Column': prediction_columns,
        'Precision (Avg)': precision_avg_list,
        'Precision (Std)': precision_std_list,
        'Recall (Avg)': recall_avg_list,
        'Recall (Std)': recall_std_list,
        'F1-score (Avg)': f1_avg_list,
        'F1-score (Std)': f1_std_list
    })

    return results_table

# Example of how to use the functions on a dataframe
# Assuming `bin_results` is your dataframe and it has a target column 'relevance_manual'
target_column = 'relevance_manual'
prediction_columns = [f'k={i}' for i in range(0, 6)]
results_table1 = generate_results_table(total_test, 'relevance_PS_manual', ['k=0', 'k=1', 'k=5', 'previous_ann'])
print(results_table1.to_string(index=False))


In [ ]:
print(total_test[['relevance_PS_manual', 'k=0', 'k=1', 'k=5']])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_metrics(df, target_column, prediction_columns):
    # Initialize a dictionary to store the results
    results = {col: {'precision': [], 'recall': [], 'f1_score': []} for col in prediction_columns}

    # Loop over each prediction column
    for col in prediction_columns:
        # Get the ground truth and predictions, ignoring NaN values
        y_true = df[target_column]
        y_pred = df[col].dropna()
        y_true = y_true.loc[y_pred.index]

        # Calculate precision, recall, and F1-score
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)

        # Append the scores to the results dictionary
        results[col]['precision'].append(precision)
        results[col]['recall'].append(recall)
        results[col]['f1_score'].append(f1)

    return results

def generate_results_table(df, target_column, prediction_columns):
    # Calculate metrics
    results = calculate_metrics(df, target_column, prediction_columns)

    # Calculate averages and standard deviations for each prediction column
    precision_avg_list, precision_std_list = [], []
    recall_avg_list, recall_std_list = [], []
    f1_avg_list, f1_std_list = [], []

    for col in prediction_columns:
        precision_avg = np.mean(results[col]['precision'])
        precision_std = np.std(results[col]['precision'])
        recall_avg = np.mean(results[col]['recall'])
        recall_std = np.std(results[col]['recall'])
        f1_avg = np.mean(results[col]['f1_score'])
        f1_std = np.std(results[col]['f1_score'])

        precision_avg_list.append(precision_avg)
        precision_std_list.append(precision_std)
        recall_avg_list.append(recall_avg)
        recall_std_list.append(recall_std)
        f1_avg_list.append(f1_avg)
        f1_std_list.append(f1_std)

    # Create a DataFrame to store the results
    results_table = pd.DataFrame({
        'Prediction Column': prediction_columns,
        'Precision (Avg)': precision_avg_list,
        'Precision (Std)': precision_std_list,
        'Recall (Avg)': recall_avg_list,
        'Recall (Std)': recall_std_list,
        'F1-score (Avg)': f1_avg_list,
        'F1-score (Std)': f1_std_list
    })

    return results_table

# Example of how to use the functions on a dataframe
# Assuming `bin_results` is your dataframe and it has a target column 'relevance_manual'
target_column = 'relevance_manual'
prediction_columns = [f'k={i}' for i in range(0, 6)]
results_table1 = generate_results_table(bin_results, 'relevance_manual', ['k=0', 'k=1', 'k=5', 'k=2'])
print(results_table1.to_string(index=False))


In [ ]:
#print(bin_results['relevance_manual'])
#print(bin_results['k=2'])


from sklearn.metrics import precision_score, recall_score, f1_score

y_true = list(bin_results['relevance_manual'])
y_pred = list(bin_results['k=2'] + 1)

precision = (precision_score(y_true, y_pred))
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(precision, recall, f1)

In [ ]:
print(type(bin_results['relevance_manual'][0]))
print(type(bin_results['k=2'][0]))


In [ ]:
print(len(bin_results['relevance_manual']))
print(len(bin_results['k=2']))
